<a href="https://colab.research.google.com/github/palanipsb/Chatbot/blob/master/TrainTicketBookingWithAdmin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [230]:
class ConfigLables:

  GETTRAINNUMBER = '\nPlease Enter the Train Number: '
  BOOKINGCONFIRM = '\nCongrats! Booking confirmed!. Please note the PNR number to check booking details. PNR:'
  BOOKINGCACEL = 'Thank you!. Your ticket has been cancelled. You will get your refund in 48 hours!'
  BOOKINGDETAILS = '\n'+'*'*30+' Booking Details for PNR: {} '+'*'*30
  USERNAME = '\nPlease enter booking name: '
  USERYOB = '\nPlease enter the YOB: '
  USERAGE = '\nPlease enter the Age: '
  GETTHEOPTION = '\nPlease Enter the Option: '
  GETPNRNUMBER = '\nPlease Enter the PNR Number: '
  GETCOACH = '\nPlease Enter the Coach: '
  GETSTATIONCODE = '\nPlease Enter the Station Code: '
  GETPASSANGERCOUNT = '\nPlease enter number of passangers: '
  GETFOOFQTY = '\nPlease Enter the food quatity: '
  DISPTRANDETAILS = '{0} : From - {1}: {2} To {3}: {4} (Stopping Stations: {5})\n'
  DISPCOACHDETAILS = 'Coach: {0}  || Available seats({1})  || Fare: {2:.2f} Per Adult + Reservartion Chargers'
  DISPTRAINUMBER ='\n'+'*'*30+' Train Number: {} '+'*'*30

class TrainDetails(ConfigLables):
  trainData = {}

  def __init__(
      self,
      trainNo,
      trainName,
      seatCount1AC,
      baseFare1AC,
      seatCount2AC,
      baseFare2AC,
      seatCountSl,
      baseFareSl,
      seatCountG,
      baseFareG,
      stoppings,
      deptStat,
      arrivStat,
      pricePerKm,
  ):
      self.trainNo = trainNo
      self.trainName = trainName
      self.seatCount1AC = seatCount1AC
      self.baseFare1AC = baseFare1AC
      self.seatCount2AC = seatCount2AC
      self.baseFare2AC = baseFare2AC
      self.seatCountSl = seatCountSl
      self.baseFareSl = baseFareSl
      self.seatCountG = seatCountG
      self.baseFareG = baseFareG
      self.stoppings = stoppings
      self.deptStat = deptStat
      self.arrivStat = deptStat
      self.pricePerKm = pricePerKm

  def trainDataEntry(self):
      self.trainData.update(
          {
              self.trainNo: {
                  "name": self.trainName,
                  "availableCoaches": {
                      "1AC": {
                          "availableSeats": self.seatCount1AC,
                          "baseFare": self.baseFare1AC,
                      },
                      "2AC": {
                          "availableSeats": self.seatCount2AC,
                          "baseFare": self.baseFare2AC,
                      },
                      "SLP": {
                          "availableSeats": self.seatCountSl,
                          "baseFare": self.baseFareSl,
                      },
                      "GEN": {
                          "availableSeats": self.seatCountG,
                          "baseFare": self.baseFareG,
                      },
                  },
                  "stoppingSations": self.stoppings,
                  "departureStation": self.deptStat,
                  "arrivalStation": self.arrivStat,
                  "pricePerKm": self.pricePerKm,
              }
          }
      )

  def getTrainName(self, trainNo):
    return [j['name'] for i, j in self.trainData.items() if i==self.trainNo][0]

  def getTrainStoppings(self,trainNo):
    return [j['stoppingSations'] for i, j in self.trainData.items() if i==self.trainNo][0]

  def getCoachDetails(self,trainNo):
    self.coachOptions = {}
    for i,j in self.trainData.items():
      if i==self.trainNo:
        for a,b in j['availableCoaches'].items():
          self.coachOptions.update({a:{'seats':b['availableSeats'], 'baseBare':b['baseFare']}})
    return(self.coachOptions)

  def updateAvailableSeats(self,trainNo,coach,numberOfPassangers,bookOrCancel):
    self.coach = coach
    self.numberOfPassangers = numberOfPassangers
    self.bookOrCancel = bookOrCancel
    for i,j in self.trainData.items():
      if i==self.trainNo:
        for a,b in j['availableCoaches'].items():
          if a==self.coach:
            if self.bookOrCancel == 'Cancel':
              self.trainData[i]['availableCoaches'][a]['availableSeats'] -= self.numberOfPassangers
            else:
              self.trainData[i]['availableCoaches'][a]['availableSeats'] += self.numberOfPassangers

  def displayTrainDetailsWithCoach(self,trainNo):
    #self.coachOptions = {}
    print(self.DISPTRAINUMBER.format(self.trainNo))
    for i,j in self.trainData.items():
      if i==self.trainNo:
        for a,b in j['availableCoaches'].items():
          #self.coachOptions.update({a:{'seats':b['availableSeats'], 'baseBare':b['baseFare']}})
          print(self.DISPCOACHDETAILS.format(a,b['availableSeats'],b['baseFare']))
    #return self.coachOptions

  def displayTrainDetails(self,trainNo):
    self.stoppingSats = []
    for i,j in TrainDetails.trainData.items():
      if i==self.trainNo:
        for m,n in TrainDetails.stationData.items():
          if m == j['departureStation']:
            self.depatureStationCode,self.depatureStationName = m,n['name']
          elif m == j['arrivalStation']:
            self.arrivalStationCode,self.arrivalStationName = m,n['name']
          if m in j['stoppingSations']:
            self.stoppingSats = self.stoppingSats + m + ' - '+n['name']+'; '
        print(self.DISPTRANDETAILS.format(self.trainNo,self.depatureStationCode,self.depatureStationName,self.arrivalStationCode,self.arrivalStationName,self.stoppingSats[0:-2]))

class StationDetails:
  stationData = {}
  stoppingTrains = {}

  def __init__(self, stationCode, stationName, distance, trainDetails):
      self.stationCode = stationCode
      self.stationName = stationName
      self.distance = distance
      self.trainDetails = trainDetails

  def stationDataEntry(self):
      stationData = {}
      for i, j in self.trainDetails.items():
          self.stoppingTrains.update(
              {
                  i: {
                      "arrivalTime": j["arrivalTime"],
                      "departureTime": j["departureTime"],
                  }
              }
          )
          StationDetails.stationData.update(
              {
                  self.stationCode: {
                      "name": self.stationName,
                      "distance": self.distance,
                      "stoppingTrainNo": self.stoppingTrains,
                  }
              }
          )


class FoodDetails:
  foodData = {}

  def __init__(self, trainNo, foodDetails):
      self.trainNo = trainNo
      self.foodDetails = foodDetails

  def foodDataEntry(self):
      self.cuisineList = {}
      for i, j in self.foodDetails.items():
          self.subFoodList = {}
          for k, l in j["mealOptions"].items():
              self.subFoodList.update(
                  {
                      k: {
                          "meal": l["meal"],
                          "availableQty": l["availableQty"],
                          "price": l["price"],
                      }
                  }
              )
          self.cuisineList.update(
              {i: {"cuisine": j["cuisine"], "mealOptions": self.subFoodList}}
          )
      self.foodData.update({self.trainNo: self.cuisineList})

class TicketHandling:

  pass


class UserPrompt:

  def userResponse(self,promptText, inputType, inputVerifier):
    self.promptText = promptText
    self.inputType = inputType
    self.inputVerifier = inputVerifier
    if self.inputType == int:
      self.userResponse = int(input(self.promptText))
    else:
      self.userResponse = input(self.promptText)
    while self.userResponse not in self.inputVerifier:
      if self.inputType == int:
        self.userResponse = int(UserPrompt.userResponse(self,self.promptText, self.inputType, self.inputVerifier))
      else:
        self.userResponse = UserPrompt.userResponse(self,self.promptText, self.inputType, self.inputVerifier)
    return self.userResponse





In [133]:
class ConfigLables:

  GETTRAINNUMBER = '\nPlease Enter the Train Number: '

class UserPrompt(ConfigLables):

  def userResponse(self,promptText, inputType, inputVerifier):
    self.promptText = promptText
    self.inputType = inputType
    self.inputVerifier = inputVerifier
    if self.inputType == int:
      self.userResponse = int(input(self.promptText))
    else:
      self.userResponse = input(self.promptText)
    while self.userResponse not in self.inputVerifier:
      if self.inputType == int:
        self.userResponse = int(UserPrompt.userResponse(self,self.promptText, self.inputType, self.inputVerifier))
      else:
        self.userResponse = UserPrompt.userResponse(self,self.promptText, self.inputType, self.inputVerifier)
    return self.userResponse

In [52]:
u1 = UserPrompt()
u1.userResponse(u1.GETTRAINNUMBER,str,['100001','100002'])


Please Enter the Train Number: 1001

Please Enter the Train Number: 100001


'100001'

In [231]:
t1 = TrainDetails(100001,'Madurai Express',100,50.00,150,40.00,300,30.00,140,10.00,['VM','TPJ'],'MS','MDU',2.00)
t1.trainDataEntry()

In [235]:
t1.displayTrainDetailsWithCoach(100001)


****************************** Train Number: 100001 ******************************
Coach: 1AC  || Available seats(200)  || Fare: 50.00 Per Adult + Reservartion Chargers
Coach: 2AC  || Available seats(150)  || Fare: 40.00 Per Adult + Reservartion Chargers
Coach: SLP  || Available seats(300)  || Fare: 30.00 Per Adult + Reservartion Chargers
Coach: GEN  || Available seats(140)  || Fare: 10.00 Per Adult + Reservartion Chargers


In [234]:
t1.updateAvailableSeats(100001,'1AC',150,'book')

In [125]:
print(t1.getTrainName(100001))

Madurai Express


In [ ]:
s1 = StationDetails('MS','Chennai Egmore',0,{100001:{'arrivalTime':'10.00 AM','departureTime':'10:10 AM'},100004:{'arrivalTime':'12:00 PM', 'departureTime':'12:10 PM'}})
s1.stationDataEntry()

In [ ]:
f1 = FoodDetails(100001,{1:{'cuisine':'South Indian','mealOptions': {1:{'meal':'Break Fast - Idly', 'availableQty':50,'price':50},
                                                      2:{'meal':'Lunch - Biriyani', 'availableQty':50,'price':50}}},
                         2:{'cuisine':'North Indian','mealOptions': {1:{'meal':'Break Fast - Poori', 'availableQty':50,'price':50},
                                                      2:{'meal':'Lunch - Chapathi', 'availableQty':50,'price':50}}},})
f1.foodDataEntry()
FoodDetails.foodData

{100001: {1: {'cuisine': 'South Indian',
   'mealOptions': {1: {'meal': 'Break Fast - Idly',
     'availableQty': 50,
     'price': 50},
    2: {'meal': 'Lunch - Biriyani', 'availableQty': 50, 'price': 50}}},
  2: {'cuisine': 'North Indian',
   'mealOptions': {1: {'meal': 'Break Fast - Poori',
     'availableQty': 50,
     'price': 50},
    2: {'meal': 'Lunch - Chapathi', 'availableQty': 50, 'price': 50}}}}}

In [ ]:
testData = {100001:{1:{'cuisine':'South Indian',1:{'meal':'Break Fast - Idly','availableQty':50,'price':50.00}}}}
testData

{100001: {1: {'cuisine': 'South Indian',
   1: {'meal': 'Break Fast - Idly', 'availableQty': 50, 'price': 50.0}}}}

In [ ]:
TranDetails.trainData

{100001: {'name': 'Madurai Express',
  'availableCoaches': {'1AC': {'availableSeats': 100, 'baseFare': 50.0},
   '2AC': {'availableSeats': 150, 'baseFare': 40.0},
   'SL': {'availableSeats': 300, 'baseFare': 30.0},
   'GN': {'availableSeats': 140, 'baseFare': 10.0}},
  'stoppingSations': ['VM', 'TPJ'],
  'departureStation': 'MS',
  'arrivalStation': 'MS',
  'pricePerKm': 2.0}}

In [ ]:
StationDetails.stationData

{'MS': {'name': 'Chennai Egmore',
  'distance': 0,
  'stoppingTrainNo': {100001: {'arrivalTime': '10.00 AM',
    'departureTime': '10:10 AM'},
   100004: {'arrivalTime': '12:00 PM', 'departureTime': '12:10 PM'}}}}